# Machine Learning Homework 1

### Task 2

MultiBoosting 算法[2] 将AdaBoost 作为Bagging 的基学习器，Iterative Bagging 算法[1] 则是将Bagging 作为AdaBoost 的基学习器。试比较二者的优缺点。

Author: Zhilong Hong

In [1]:
# Import part
import time
import pandas as pd
from scipy.stats import uniform

from multiboost import MultiBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from evaluation import ClassifierEvaluator

In [2]:
# Read data from file

def convert_rank_to_int(rank):
    return ord(rank) - 96


f = open("./dataset/krkopt.data", 'r')
data_set = []
for line in f.readlines():
    data = line.strip().split(',')
    data_set.append({
        "white-king-file": convert_rank_to_int(data[0]),
        "white-king-rank": data[1],
        "white-rook-file": convert_rank_to_int(data[2]),
        "white-root-rank": data[3],
        "black-king-file": convert_rank_to_int(data[4]),
        "black-king-rank": data[5],
        "optimal-depth-of-win": data[6]
    })
data_set = pd.DataFrame.from_dict(data_set)

In [3]:
# Prepare dataset
#data_set=data_set[data_set['optimal-depth-of-win'].isin(['fourteen','draw'])]

x = data_set[[col for col in data_set.columns if col not in ["optimal-depth-of-win"]]]
y = data_set[["optimal-depth-of-win"]].values.reshape((-1))
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [4]:
%%time

clf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=None),n_estimators=100)
clf.fit(x_train, y_train)
score = accuracy_score(y_test, clf.predict(x_test))
print(score)

0.844262295082
Wall time: 120 ms


In [5]:
%%time

clf = MultiBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=None), n_estimators=100)
clf.fit(x_train, y_train)
score = accuracy_score(y_test, clf.predict(x_test))
print(score)

0.870990734141
Wall time: 6.86 s


In [6]:
%%time

clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=None), n_estimators=100)
clf.fit(x_train, y_train)
score = accuracy_score(y_test, clf.predict(x_test))
print(score)

0.868852459016
Wall time: 2.67 s


In [6]:
clf = MultiBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=None), n_estimators=100)
e = ClassifierEvaluator(clf)
e.fit(x, y)
print('Bias:', e.bias(), 'Var:', e.var())

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [03:37<00:00, 21.70s/it]


Bias: 0.151714736282 Var: 2.22779479009e-06


In [7]:
clf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=None), n_estimators=100)
e = ClassifierEvaluator(clf)
e.fit(x, y)
print('Bias:', e.bias(), 'Var:', e.var())

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  3.70it/s]


Bias: 0.187108527382 Var: 5.43669471653e-06


In [8]:
clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=None), n_estimators=100)
e = ClassifierEvaluator(clf)
e.fit(x, y)
print('Bias:', e.bias(), 'Var:', e.var())

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:11<00:00,  7.19s/it]


Bias: 0.153726299319 Var: 5.31041334203e-06
